In [25]:
import os.path as op
import numpy as np
import numpy.matlib
from pandas import read_csv
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.manifold import MDS

import mne
from mne.io import read_raw_fif, concatenate_raws
from mne.datasets import visual_92_categories

import gc
import scipy
from scipy import stats

In [2]:
print(__doc__)

data_path = visual_92_categories.data_path()

# Define stimulus - trigger mapping
fname = op.join(data_path, 'visual_stimuli.csv')
conds = read_csv(fname)
print(conds.head(5))

Automatically created module for IPython interactive environment
   trigger       condition  human  face  animal  natural
0        0  human bodypart      1     0       1        1
1        1  human bodypart      1     0       1        1
2        2  human bodypart      1     0       1        1
3        3  human bodypart      1     0       1        1
4        4  human bodypart      1     0       1        1


In [3]:
max_trigger = 92
conds = conds[:max_trigger]  # take the first 92 rows

In [4]:
conditions = []
for c in conds.values:
    cond_tags = list(c[:2])
    cond_tags += [('not-' if i == 0 else '') + conds.columns[k]
                  for k, i in enumerate(c[2:], 2)]
    conditions.append('/'.join(map(str, cond_tags)))
#print(conditions[:])

In [5]:
event_id = dict(zip(conditions, conds.trigger + 1))
#event_id['0/human bodypart/human/not-face/animal/natural']

In [6]:
n_runs = 1  # 4 for full data (use less to speed up computations)
fname = op.join(data_path, 'sample_subject_%i_tsss_mc.fif') # only one subject
raws = [read_raw_fif(fname % block, verbose='error')
        for block in range(n_runs)]  # ignore filename warnings
raw = concatenate_raws(raws)

events = mne.find_events(raw, min_duration=.002)
events = events[events[:, 2] <= max_trigger]

1374 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93 200 222 244]


In [7]:
picks = mne.pick_types(raw.info, meg=True)
epochs = mne.Epochs(raw, events=events, event_id=event_id, baseline=None,
                    picks=picks, tmin=-.1, tmax=.4, preload=True) # epochs shape is (#epochs, #channels, #times)
del raw

Not setting metadata
Not setting metadata
920 matching events found
No baseline correction applied
0 projection items activated
Loading data for 920 events and 501 original time points ...
0 bad epochs dropped


In [ ]:
epochs.plot(n_epochs=5)

In [10]:
ep50_300 = epochs.copy().crop(0.05, 0.3).get_data() # X
base10_0 = epochs.copy().crop(-0.1, 0).get_data() # Q

In [11]:
print(ep50_300.shape)
print(base10_0.shape)

(920, 306, 251)
(920, 306, 101)


In [12]:
# Morlet Wavelet
myfreqrange = np.array([2,120])
Fs = 1000; # Hz, sampling rate
# timedur = ep50_300.shape[2]/Fs # seconds, temporal duration
timevec = np.linspace(0.05, 0.3, 251) # vector of time between 0 and timedur seconds
timevec_gauss = np.linspace(-2, 2, 4001)
lofreq = myfreqrange[0]
hifreq = myfreqrange[1]

NWavelets = 236
NCycles = 7
MyFreqs = np.linspace(lofreq, hifreq, NWavelets)
MyMorletFamily = np.empty((0, 4001), float) # equivalent to "clear"

In [13]:
def normpdf(x, mu=0, sigma=1):
    u = float((x-mu) / abs(sigma))
    y = np.exp(-u*u/2) / (np.sqrt(2*np.pi) * abs(sigma))
    return y

# Wavelet family
for wnum in range(0, NWavelets):
        myomega = 2 * np.pi * MyFreqs[wnum];
        mysigma = NCycles/myomega
        mygauss = np.array([normpdf(i,0,mysigma)
                    for i in timevec_gauss])
        mySig = np.exp(1j*myomega*timevec_gauss)
        MyMorletFamily = np.append(MyMorletFamily, [mySig * mygauss], axis = 0)

In [14]:
print(MyMorletFamily.shape)
ep50_300.shape[1]

(236, 4001)


306

In [15]:
def fre_decomp(fftMEG, MyMorletFamily, NEvents, nconvolution, nshift):
    fftGW = np.fft.fft(MyMorletFamily[:], nconvolution) # one dimensional transform?
    fftconv = fftMEG * np.matlib.repmat(fftGW, NEvents, 1) # * for multiplying numpy arrays
    conv_result = np.fft.ifft(fftconv, nconvolution,1)
    conv_result = conv_result[:, (nshift):(conv_result.shape[1] - nshift)] # index?  
    power = np.power(np.absolute(conv_result), 2)
    del fftGW 
    del fftconv
    del conv_result
    gc.collect()
    return power

In [62]:
IDX = np.zeros((25))
IDX
x = timevec
y = TimeBins
line = 1
distances = np.abs(x - y[line])
distances.argmin()
TimeBins.shape[0]

# implements Matlab dsearchn
def dsearchn(x,y):
    IDX = np.zeros((y.shape[0]), int)
    for line in range(0, y.shape[0]):
        distances = np.abs(x - y[line])
        distances.argmin()
        IDX[line] = distances.argmin().astype(int)
    return IDX

In [20]:
# testing for data storage methods

# dtype obj approach
x = np.zeros(16, dtype = [('MEG_Power',int),('data',[]), ('MyFreqs',[]), 
                                ('TBins',[]), ('FBins',[]), ('EEGFeatures',[])])

m = np.dtype([('name', np.unicode_, 16), ('grades', np.float64, (2,))])
y = 1
# print(type(m))
# print(type(x))
# print(type(x[0]))
# print(x)
# #x[0].MEG_Power = np.array(y)


# nested dict approach
testdata = {}
testpower = np.array([1,2,3,4])#np.zeros((1,2,3,4))
                    
for subj in range(0,16):
    testdata[subj] = {}
    for chan in (0,2):
        testdata[subj]['MEG_Power'] = testpower
        testdata[subj]['bins'] = 1
        testdata[subj]['strdata'] = "string"
        
#print("dictionary of subjs 1-16:")
#print(testdata)

#print("data for subj 1:")
#print(testdata[0])

print("data for subj 1, attribute 'strdata':")
print(testdata[0]['MEG_Power'].shape)


data for subj 1, attribute 'strdata':
(4,)


In [63]:
def FeatureBins(MEG_chan,TimeInds, FreqInds, NEvents):
    NTBins = len(TimeInds)
    NFBins = len(FreqInds)
    this_featureset = np.zeros((NEvents, NFBins-1, NTBins-1))
    for time in range (0, NTBins-1):
        for freq in range (0, NFBins-1):
            temp = MEG_chan[(FreqInds[freq]):(FreqInds[freq + 1]), (TimeInds[time]):(TimeInds[time + 1]), :]
            # remove z-score if not operating w in participants-- MUST HAVE
            # RIGHT UNIT OF SELECTION (from Spezio)
            this_featureset[:, freq, time] = stats.zscore(np.median(np.squeeze(np.median(temp,0)),0)) # need py equivalent for: 
            # = zscore(median(squeeze(median(temp,1)),1))
     
    return this_featureset
#     AllData(subj).EEGFeatures = this_featureset


In [68]:
# for transferring time series data into frequency domain
MEG_Power = np.zeros((920,306,8,25))

# Baseline; divide by -.1 to 0


# Time Bins
TimeBins = np.linspace(0.05, 0.3, 26)

# Frequency Bins
FreqBins = np.array([2, 4, 8, 13, 20, 35, 55, 80, 120])

Nchan = ep50_300.shape[1]
#NTBins = len(TimeBins)
#NFBins = len(FreqBins)
nshift = int((timevec_gauss.size-1)/2)

In [66]:
# works but extremely slow

# # initialize dtype array for data storage 
# # 16 participants are indexes, with fields given by dtype
# # access values like AllData[2].MEG_Power = MEG_Power struct for participant 3
# # i think
# AllData = np.zeros(16, dtype = [('MEG_Power',[]),('data',[]), ('MyFreqs',[]), 
#                                 ('TBins',[]), ('FBins',[]), ('EEGFeatures',[])])


# initialize dict for data storage
#alldata_dict = {}

# loop thru participants first (currently only working w 1)
#for subj in range (0, 1):
    
    # create nested dict for each subj
    #alldata_dict[subj] = {}
TimeInds = dsearchn(timevec, TimeBins)
FreqInds = dsearchn(MyFreqs, FreqBins)


for chan in range(0, 15): # loop thru channels

    # Signal Data
    data = np.squeeze(ep50_300[:, chan, :])
    NEvents = data.shape[0]
    nconvolution = timevec_gauss.size + data.shape[1] - 1

    # Baseline Data
    pretrial_data = np.squeeze(base10_0[:, chan, :])
    pretrial_nconvolution = timevec_gauss.size + pretrial_data.shape[1] - 1

        #for enum in range(0, NEvents): # loop thru events

    # Convert signal & pretrial to frequency domain
    fftMEG = np.fft.fft(data[:,:], nconvolution)
    pretrial_fftMEG = np.fft.fft(pretrial_data[:,:], pretrial_nconvolution)

    MEG_chan =  np.empty((0, 251, 920))  

    for wnum in range(0, NWavelets): # loop thru wavelets
        #print(wnum)

        #fftGW = np.fft.fft(MyMorletFamily[wnum,:],nconvolution) # one dimensional transform?
        #pretrial_fftGW = np.fft.fft(MyMorletFamily[wnum,:], pretrial_nconvolution)

        #pretrial_fftconv = pretrial_fftMEG * np.matlib.repmat(pretrial_fftGW,NEvents,1)
        #pretrial_conv_result = np.fft.ifft(pretrial_fftconv,pretrial_nconvolution,1)
        #pretrial_conv_result = pretrial_conv_result[:,(nshift):(pretrial_conv_result.shape[1] - nshift)]
        #pretrial_power = np.power(np.absolute(pretrial_conv_result), 2)

        #fftconv = fftMEG * np.matlib.repmat(fftGW,NEvents,1) # * for multiplying numpy arrays
        #conv_result = np.fft.ifft(fftconv,nconvolution,1)
        #conv_result = conv_result[:,(nshift):(conv_result.shape[1] - nshift)] # index? 

        pretrial_power = fre_decomp(pretrial_fftMEG, MyMorletFamily[wnum,:], NEvents, pretrial_nconvolution, nshift)
        signal_power = fre_decomp(fftMEG, MyMorletFamily[wnum,:], NEvents, nconvolution, nshift)

        normalized_MEG = np.transpose(signal_power)/np.matlib.repmat(np.median(np.transpose(pretrial_power), 0) , 251, 1)

        MEG_chan = np.append(MEG_chan, [normalized_MEG], axis = 0) # numpy equivalent of transpose
        
    MEG_chan_Bins = FeatureBins(MEG_chan, TimeInds, FreqInds ,NEvents) # Feature set for the current electrode
    # electrode x trial x wavelet x time
    MEG_Power = np.append(MEG_Power, [MEG_chan_Bins], axis = 0)
    
    del MEG_chan
    del MEG_chan_Bins
    gc.collect() # clean memory      
    print(chan)
# timevec, TimeBins, MyFreqs, FreqBins were transposed in Matlab example
# unsure if this works


0
1


In [71]:
MEG_Power.shape

(920, 306, 8, 25)

In [ ]:
TimeInds = dsearchn(timevec, TimeBins)
FreqInds = dsearchn(MyFreqs, FreqBins)

NTBins = len(TimeInds)
NFBins = len(FreqInds)

alldata_dict[subj]['MEG_Power'] = MEG_Power
alldata_dict[subj]['MyFreqs'] = MyFreqs
alldata_dict[subj]['TBins'] = TimeBins
alldata_dict[subj]['FBins'] = FreqBins

#     AllData[subj].MEG_Power = MEG_Power
#     AllData[subj.MyFreqs = MyFreqs
#     AllData[subj].TBins = TimeBins
#     AllData[subj].FBins = FreqBins


# need to replace obj.shape:
#temp = np.zeros(temp.shape)
#this_featureset = np.zeros(this_featureset.shape)

# realllly unsure if this works
for time in range (0, NTBins-1):
    for freq in range (0, NFBins-1):
        temp = MEG_Power[:, (FreqInds[freq]):(FreqInds[freq + 1]), 
                         (TimeInds[time]):(TimeInds[time + 1]), :]
        # remove z-score if not operating w in participants-- MUST HAVE
        # RIGHT UNIT OF SELECTION (from Spezio)
        this_featureset[freq,time,:] # need py equivalent for: 
        # = zscore(median(squeeze(median(temp,1)),1))
     
    alldata_dict[subj]['EEGFeatures'] = this_featureset
#     AllData(subj).EEGFeatures = this_featureset

    
#--------------------------------
                #MEG_dB(wnum,:,:) = 10*log10(MEG_Power(wnum,:,:))

                #baseline???
                #for loops??
                #np.fft.fft
                # For numpy array, it seems that * is an element wise multiplication operator. But are we operating on np.array?


#fast fourier transform (in numpy) (cohen's book) np.fft


#one participant, for now  
#wavelet number, then channel number


#fft for each channel
# then for each wavelet, going to do the fft of the gabor wavelet family, organize it by channel
#element by element multiplication in python: numpy dot multiply
#conv result using ifft , second dimension (,1) in python
#nshift gives center
#then baseline correct the power; conv result from 50-300 and divide from -.1 to 0

In [229]:
# works but extremely slow

# # initialize dtype array for data storage 
# # 16 participants are indexes, with fields given by dtype
# # access values like AllData[2].MEG_Power = MEG_Power struct for participant 3
# # i think
# AllData = np.zeros(16, dtype = [('MEG_Power',[]),('data',[]), ('MyFreqs',[]), 
#                                 ('TBins',[]), ('FBins',[]), ('EEGFeatures',[])])


# initialize dict for data storage
alldata_dict = {}

# loop thru participants first (currently only working w 1)
for subj in range (0, 1):
    
    # create nested dict for each subj
    alldata_dict[subj] = {}

    for chan in range(0, 1): # loop thru channels

        # Signal Data
        data = np.squeeze(ep50_300[:, chan, :])
        NEvents = data.shape[0]
        nconvolution = timevec_gauss.size + data.shape[1] - 1

        # Baseline Data
        pretrial_data = np.squeeze(base10_0[:, chan, :])
        pretrial_nconvolution = timevec_gauss.size + pretrial_data.shape[1] - 1

            #for enum in range(0, NEvents): # loop thru events

        # Convert signal & pretrial to frequency domain
        fftMEG = np.fft.fft(data[:,:], nconvolution)
        pretrial_fftMEG = np.fft.fft(pretrial_data[:,:], pretrial_nconvolution)

        MEG_chan =  np.empty((0, 251, 920))  

        for wnum in range(0, NWavelets): # loop thru wavelets
            print(wnum)

            #fftGW = np.fft.fft(MyMorletFamily[wnum,:],nconvolution) # one dimensional transform?
            #pretrial_fftGW = np.fft.fft(MyMorletFamily[wnum,:], pretrial_nconvolution)

            #pretrial_fftconv = pretrial_fftMEG * np.matlib.repmat(pretrial_fftGW,NEvents,1)
            #pretrial_conv_result = np.fft.ifft(pretrial_fftconv,pretrial_nconvolution,1)
            #pretrial_conv_result = pretrial_conv_result[:,(nshift):(pretrial_conv_result.shape[1] - nshift)]
            #pretrial_power = np.power(np.absolute(pretrial_conv_result), 2)

            #fftconv = fftMEG * np.matlib.repmat(fftGW,NEvents,1) # * for multiplying numpy arrays
            #conv_result = np.fft.ifft(fftconv,nconvolution,1)
            #conv_result = conv_result[:,(nshift):(conv_result.shape[1] - nshift)] # index? 

            pretrial_power = fre_decomp(pretrial_fftMEG, MyMorletFamily[wnum,:], NEvents, pretrial_nconvolution, nshift)
            signal_power = fre_decomp(fftMEG, MyMorletFamily[wnum,:], NEvents, nconvolution, nshift)

            normalized_MEG = np.transpose(signal_power)/np.matlib.repmat(np.median(np.transpose(pretrial_power), 0) , 251, 1)

            MEG_chan = np.append(MEG_chan, [normalized_MEG], axis = 0) # numpy equivalent of transpose
       
        # electrode x wavelet x time x trial
        MEG_Power = np.append(MEG_Power, [MEG_chan], axis = 0)
        
        del MEG_chan
        gc.collect() # clean memory      

    # timevec, TimeBins, MyFreqs, FreqBins were transposed in Matlab example
    # unsure if this works
    TimeInds = dsearchn(timevec, TimeBins)
    FreqInds = dsearchn(MyFreqs, FreqBins)

    NTBins = len(TimeInds)
    NFBins = len(FreqInds)
    
    alldata_dict[subj]['MEG_Power'] = MEG_Power
    alldata_dict[subj]['MyFreqs'] = MyFreqs
    alldata_dict[subj]['TBins'] = TimeBins
    alldata_dict[subj]['FBins'] = FreqBins

#     AllData[subj].MEG_Power = MEG_Power
#     AllData[subj.MyFreqs = MyFreqs
#     AllData[subj].TBins = TimeBins
#     AllData[subj].FBins = FreqBins
  

    # need to replace obj.shape:
    #temp = np.zeros(temp.shape)
    #this_featureset = np.zeros(this_featureset.shape)
    
    # realllly unsure if this works
    for time in range (0, NTBins-1):
        for freq in range (0, NFBins-1):
            temp = MEG_Power[:, (FreqInds[freq]):(FreqInds[freq + 1]), 
                             (TimeInds[time]):(TimeInds[time + 1]), :]
            # remove z-score if not operating w in participants-- MUST HAVE
            # RIGHT UNIT OF SELECTION (from Spezio)
            this_featureset[freq,time,:] # need py equivalent for: 
            # = zscore(median(squeeze(median(temp,1)),1))
     
    alldata_dict[subj]['EEGFeatures'] = this_featureset
#     AllData(subj).EEGFeatures = this_featureset

    
#--------------------------------
                #MEG_dB(wnum,:,:) = 10*log10(MEG_Power(wnum,:,:))

                #baseline???
                #for loops??
                #np.fft.fft
                # For numpy array, it seems that * is an element wise multiplication operator. But are we operating on np.array?


#fast fourier transform (in numpy) (cohen's book) np.fft


#one participant, for now  
#wavelet number, then channel number


#fft for each channel
# then for each wavelet, going to do the fft of the gabor wavelet family, organize it by channel
#element by element multiplication in python: numpy dot multiply
#conv result using ifft , second dimension (,1) in python
#nshift gives center
#then baseline correct the power; conv result from 50-300 and divide from -.1 to 0

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235


NameError: name 'this_featureset' is not defined

In [232]:
print(alldata_dict[0]['MEG_Power'].shape)

(1, 236, 251, 920)


In [ ]:
# Classify using the average signal in the window 50ms to 300ms
# to focus the classifier on the time interval with best SNR.
clf = make_pipeline(StandardScaler(),
                    LogisticRegression(C=1, solver='liblinear',
                                       multi_class='auto'))

#y = epochs.events[:, 2]
y_sup = (epochs.events[:, 2] > 48).astype(int) #set up superordinate classification label. 
classes = set(y_sup) 

#cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

# Compute confusion matrix for each cross-validation fold
#y_pred = np.zeros((len(y_sup), len(classes)))
#for train, test in cv.split(ep50_300, y_sup):
    # Fit
 #   clf.fit(ep50_300[train], y_sup[train])
    # Probabilistic prediction (necessary for ROC-AUC scoring metric)
  #  y_pred[test] = clf.predict_proba(ep50_300[test])

In [14]:
y_sup = (epochs.events[:, 2] > 48).astype(int)
y_sup.shape
classes

NameError: name 'classes' is not defined

In [ ]:
# Train-test split

In [ ]:
# Regularized Regression Lasso / Ridge / Logistic

In [ ]:
# SVM

In [ ]:
# RVM

In [ ]:
# RF

In [ ]:
# PCA

In [90]:
confusion = np.zeros((len(classes), len(classes)))
for ii, train_class in enumerate(classes):
    for jj in range(ii, len(classes)):
        confusion[ii, jj] = roc_auc_score(y_sup == train_class, y_pred[:, jj])
        confusion[jj, ii] = confusion[ii, jj]
confusion  

array([[0.67340695, 0.32659305],
       [0.32659305, 0.67340695]])